<a href="https://colab.research.google.com/github/nkrumahthis/alu-x-meta-stitches/blob/main/stitches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform openai google

In [ ]:
import openai

from google.auth import default, transport
from google.cloud import storage

from google.oauth2 import service_account
from google.colab import auth

from PIL import Image
import requests

import io
from IPython.display import display

import base64

import re
import json

# path to google oauth2 JSON key file
KEY_FILE_PATH = './cred.json'

# Load credentials from the JSON key file
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE_PATH,
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

# Authenticate using the service account credentials
auth.authenticate_user()

credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

LOCATION = "us-central1"
MAAS_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
PROJECT_ID="studious-plate-436913-c1"
MODEL_ID="meta/llama-3.2-90b-vision-instruct-maas"
BUCKET_URI = "gs://metallama3bootcamp"

In [ ]:
openai_client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

bucket_name = "metallama3bootcamp"

storage_client = storage.Client()

# Get the bucket
bucket = storage_client.bucket(bucket_name)


In [ ]:
blobs = bucket.list_blobs()
# Create a list of blob names
blob_names = [blob.name for blob in blobs]

image_names = [name for name in blob_names if name.endswith('.png')]
image_names.sort()

# print(blob_names)
print(image_names)

In [ ]:
river_date_list = []

rivers = {}

for image_name in image_names:
    match = re.match(r'^(.+?)_(\d{4}-\d{2}-\d{2})\.png$', image_name)
    if match:
        river_name = match.group(1)
        date = match.group(2)
        river_date_list.append({"name": river_name, "date": date, "file": image_name})

        if river_name not in rivers:
            rivers[river_name] = []
        rivers[river_name].append({"name": river_name, "date": date, "file": image_name})


print(rivers)


In [ ]:
def stitch_images(images):
  widths, heights = zip(*(i.size for i in images))

  total_width = sum(widths)
  max_height = max(heights)

  new_im = Image.new('RGB', (total_width, max_height))

  x_offset = 0
  for im in images:
    new_im.paste(im, (x_offset, 0))
    x_offset += im.size[0]

  return new_im

In [ ]:
## pass the stitched images and prompts into openai_client

def query(stitched_image, prompt):
  buffered = io.BytesIO()
  stitched_image.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue()).decode()
  river_image_url = f"data:image/png;base64,{img_str}"

  max_tokens = 200

  response = openai_client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "image_url": {
                            "url": river_image_url
                        },
                        "type": "image_url",
                    },
                    {"text": prompt, "type": "text"},
                ],
            },
        ],
        max_tokens=max_tokens,
    )

  print(response.choices[0].message.content)


def pollution_score_query(stitched_image):
  prompt = 'you are a pollution sensor. compare the parts of the stitched images and give a pollution value as a json like ```[{"pollution":1, "year": "2020-09-11"}, {"pollution":3, "year": "2020-09-11"}] pollution score is ```'
  buffered = io.BytesIO()
  stitched_image.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue()).decode()
  river_image_url = f"data:image/png;base64,{img_str}"

  max_tokens = 200

  response = openai_client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "image_url": {
                            "url": river_image_url
                        },
                        "type": "image_url",
                    },
                    {"text": prompt, "type": "text"},
                ],
            },
        ],
        max_tokens=max_tokens,
    )

  print(response.choices[0].message.content)


In [ ]:
for river in rivers.keys():
  filenames = [image['file'] for image in rivers[river]]
  filenames.sort()
  dates = [image['date'] for image in rivers[river]]
  dates.sort()

  images = []

  for filename in filenames:
    blob_name = filename
    blob = bucket.blob(blob_name)
    image_data = blob.download_as_bytes()
    image = Image.open(io.BytesIO(image_data))
    images.append(image)

  if len(images) < 2:
    continue

  stitched_image = stitch_images(images)
  prompt = ("""
    You are an environmentallist and a reporter.
    This is a stitch of satellite images of the same river %s over the years with dates %s.
    Give an analysis of the pollution and what the changes in the river are.
    """) % (river, ", ".join(dates))

  print("-"*500)
  print(river)
  # print("-"*500)
  display(stitched_image)
  query(stitched_image, prompt)
  print("="*500)
  print("\n\n\n")


